finding branching ratio between nitrate and NO2 + alkoxy under atm conditions. Numbers come from atkinson 1997

In [ ]:
import math
import numpy as np
import pandas as pd

In [ ]:
def get_nitrate_yield(carbon_number, pressure, temperature, carbon_type,
                     other_functional_groups=None):
    """
    Returns the expected fraction of ROO + NO reaction that forms RONO2
    (instead of RO + NO2).
    
    carbon number is the number of carbon atoms in the compound
    pressure is given in pascals
    temperature in K
    carbon type is a string of 'primary', 'secondary', and 'tertiary'
    other_functional_groups is a list of other functional groups that are known to
    adjust the rate. Currently accepted functional groups are:
        hydroxy group (halves the yield)
    
    
    from Carter and Atkinson 1987
    """
    # derived inputs
    concentration = pressure / 8.314 / temperature /10**6 *6.022e23 #molecule/cm3
    # diven constants
    y_inf = 0.826
    alpha = 1.94e-22 #cm3/mol-1
    beta = 0.97
    m0 = 0.0
    m_inf = 8.1
    F = 0.411
    if carbon_number < 2:
        raise AttributeError('get_nitrate_ratio is only valid for 2 or more carbons. '\
                             'See Atkinson 1997 for more information')
    # calculations
    y0 = alpha * math.exp(beta * carbon_number)
    numerator = y0*concentration * (temperature/300.)**(-m0)
    denominator = y_inf * (temperature/300.)**-m_inf
    zeta = 1 / (1+ (np.log10(numerator / denominator ))**2)
    # final answer
    rate_ratio = (numerator / (1+ numerator/denominator)) * F**zeta
    rate_yield = 1 / (1 / rate_ratio + 1) # go from ka/kb to ka/(ka+kb)
    if carbon_type == 'primary':
        rate_yield *= 0.4
    elif carbon_type == 'tertiary':
        rate_yield *= 0.3
    elif carbon_type != 'secondary':
        raise ValueError('Only primary, secondary and tertiary accepted for carbon_type')
    if other_functional_groups is not None:
        for fg in other_functional_groups:
            if (fg == 'hydroxy') or (fg == 'OH'):
                rate_yield /= 2 #from atkinson 1997 for hydroxy groups formed from alkenes
    return rate_yield

In [ ]:
# test get nitrate ratio
# using data of carter and atkinson 1989
data = [(3, 'secondary', 299, 2.37e19, 0.047),
        (4, 'secondary', 299, 2.37e19, 0.083),
        (5, 'secondary', 284, 2.52e19, 0.158),
        (6, 'secondary', 299, 2.37e19, 0.193),
        (6, 'secondary', 281, 1.20e19, 0.179),
        (3, 'primary',   299, 2.37e19, 0.019),
        (5, 'primary',   282, 2.51e19, 0.065),
        (6, 'tertiary',  298, 2.38e19, 0.059)]

for carbon_number, carbon_type, temperature, density, expected_output in data:
    pressure = density * 8.314 * temperature * 10**6 / 6.022e23
    nitrate_yield = get_nitrate_yield(carbon_number, pressure, temperature, carbon_type)
    if abs(nitrate_yield - expected_output) > 0.001:
        print('Model result different than Carter and Atkinson for data: '+str((carbon_number, carbon_type, temperature, density, expected_output)))
        print('Expected {}, got {}'.format(expected_output, nitrate_yield))

# use RMG to get plog of data

In [ ]:
from rmgpy.kinetics.arrhenius import Arrhenius, PDepArrhenius
from rmgpy.chemkin import write_kinetics_entry
from rmgpy.reaction import Reaction
from rmgpy.species import Species

In [ ]:
number_carbons = 4
carbon_type = 'primary'
peroxy_name = 'PC4H9O2'
nitrate_name = 'PC4H9ONO2'
alkoxy_name = 'PC4H9O'
peroxy_smiles = 'CCCCO[O]'
nitrate_smiles = 'CCCCO[N+](=O)[O-]'
alkoxy_smiles = 'CCCC[O]'
use_total_rate = True
other_functional_groups = ['OH']

In [ ]:
# get the alkoxy rates
if use_total_rate:
    #atkinson 1997 for C2+
    total_kinetics = Arrhenius(A=(2.7e-12*6.022e23,'cm^3/(mol*s)'),n=0,Ea=(-360*8.314,('J/mol')))
else:
    #anderlohr 2009
    alkoxy_kinetics = Arrhenius(A=(4.7e12,'cm^3/(mol*s)'),n=0,Ea=(-358,('cal/mol')))

In [ ]:
temperatures = np.linspace(250,1250,20)
pressures = np.logspace(3,7,5)

In [ ]:
data = np.ndarray((len(temperatures),len(pressures)))
data_alkoxy = np.ndarray((len(temperatures),len(pressures)))
for i1, t in enumerate(temperatures):
    for i2, p in enumerate(pressures):
        nitrate_yield = get_nitrate_yield(number_carbons,p,t,carbon_type,other_functional_groups=other_functional_groups)
        if use_total_rate:
            total_rate = total_kinetics.get_rate_coefficient(t)
            data[i1,i2] = total_rate * nitrate_yield
            data_alkoxy[i1,i2] = total_rate * (1-nitrate_yield)
        else:
            nitrate_to_NO2_ratio = 1 / (1 / nitrate_yield - 1)
            data[i1,i2] = alkoxy_kinetics.get_rate_coefficient(t) * nitrate_to_NO2_ratio

In [ ]:
nitrate_rate = PDepArrhenius().fit_to_data(temperatures,pressures, data, 'm^3/(mol*s)')
if use_total_rate:
    alkoxy_rate = PDepArrhenius().fit_to_data(temperatures,pressures, data_alkoxy, 'm^3/(mol*s)')

In [10]:
no = Species().from_smiles('[N]=O')
peroxy = Species(label = peroxy_name).from_smiles(peroxy_smiles)
nitrate = Species(label = nitrate_name).from_smiles(nitrate_smiles)

rxn = Reaction(reactants = [no,peroxy], products = [nitrate], kinetics = nitrate_rate)
print(write_kinetics_entry(rxn, [no,peroxy,nitrate]))
# this is in units of kcal/mol and mol and may need to have the activation energy modified

NO+PC4H9O2=PC4H9ONO2                                1.000e+00 0.000     0.000    
    PLOG/ 0.009869  1.704e+32 -8.087    3.870    /
    PLOG/ 0.098692  2.851e+37 -9.757    4.136    /
    PLOG/ 0.986923  5.489e+38 -10.226   3.355    /
    PLOG/ 9.869233  1.544e+38 -10.104   2.460    /
    PLOG/ 98.692327 1.031e+37 -9.762    1.765    /



In [11]:
if use_total_rate:
    alkoxy = Species(label = alkoxy_name).from_smiles(alkoxy_smiles)
    no2 = Species().from_smiles('N(=O)[O]')
    rxn2 = Reaction(reactants = [no,peroxy], products = [alkoxy, no2], kinetics = alkoxy_rate)
    print(write_kinetics_entry(rxn2, [no,peroxy,alkoxy,no2]))
    # this is in units of kcal/mol and mol and may need to have the activation energy modified

NO+PC4H9O2=PC4H9O+NO2                               1.000e+00 0.000     0.000    
    PLOG/ 0.009869  1.628e+12 -0.000    -0.715   /
    PLOG/ 0.098692  1.671e+12 -0.003    -0.709   /
    PLOG/ 0.986923  2.037e+12 -0.029    -0.670   /
    PLOG/ 9.869233  3.143e+12 -0.086    -0.586   /
    PLOG/ 98.692327 5.280e+12 -0.154    -0.489   /



In [ ]:
#nitrate
print(repr(rxn.kinetics))

In [ ]:
print(repr(rxn2.kinetics))

In [ ]:
# ensure the rates add up
errors = np.ndarray((len(temperatures),len(pressures)))
for i1, temperature in enumerate(temperatures):
    for i2, pressure in enumerate(pressures):
        if use_total_rate:
            r1 = rxn.get_rate_coefficient(temperature, pressure)
            r2 = rxn2.get_rate_coefficient(temperature, pressure)
            total = total_kinetics.get_rate_coefficient(temperature)
            error = abs(r1 + r2 - total) / total
            errors[i1, i2] = error

# compare atkinson to quantum
Here I output atkinson rates of RO2 + NO at 300, 800, and 1500 and compare it to the RRCM results by http://dx.doi.org/10.1016/j.comptc.2017.04.015

In [ ]:
import numpy as np

In [ ]:
atkinson_rxn = lambda t: 2.7e-12*np.exp(360./t)
temps = [300,800,1500]
atkinson_rates = np.array([atkinson_rxn(t) for t in temps])

In [ ]:
#these were translated by engague from figure 7 in the paper at low pressure limits.
ng_rates = np.array([2.08e-11,2.47e-11,6.58e-11])

In [ ]:
rate_diff = ng_rates/ atkinson_rates

In [ ]:
rate_diff

It seems like a different rate coefficient is predicted by the quantum and atkinson, with quantum overestimating the rate, which is typical. The rate overestimate is 2 times at atm temperature, 6 times at 800K and 20x at 1500K. 

# Compare Atkinson with Jenkins

This is to compare the results by Atkinson with those by the more recent publication of Jenkins (doi: 10.5194/acp-19-7691-2019).



In [ ]:
def get_nitrate_yield_jenkins(carbon_number, pressure, temperature, carbon_type,
                     other_functional_groups=None):
    """
    Returns the expected fraction of ROO + NO reaction that forms RONO2
    (instead of RO + NO2).
    
    carbon number is the number of carbon atoms in the compound
    pressure is given in pascals
    temperature in K
    carbon type is a string of 'primary', 'secondary', and 'tertiary'
    other_functional_groups is a list of other functional groups that are known to
    adjust the rate. Currently accepted functional groups are:
        hydroxy group (halves the yield)
    
    
    from Jenkins et al. 2019 (doi: 10.5194/acp-19-7691-2019)
    """
    # derived inputs
    concentration = pressure / 8.314 / temperature /10**6 *6.022e23 #molecule/cm3
    # diven constants
    y_inf = .43
    alpha = 2e-22 #cm3/mol-1
    beta = 1
    m0 = 0.0
    m_inf = 8
    F = 0.41
    if carbon_number < 2:
        raise AttributeError('get_nitrate_ratio is only valid for 2 or more carbons. '\
                             'See Atkinson 1997 for more information')
    # calculations
    y0 = alpha * math.exp(beta * carbon_number)
    numerator = y0*concentration * (temperature/300.)**(-m0)
    denominator = y_inf * (temperature/300.)**-m_inf
    zeta = 1 / (1+ (np.log10(numerator / denominator ))**2)
    # final answer
    rate_ratio = (numerator / (1+ numerator/denominator)) * F**zeta
    rate_yield = 1 / (1 / rate_ratio + 1) # go from ka/kb to ka/(ka+kb)
    if carbon_type == 'primary':
        rate_yield *= 0.65
    elif carbon_type == 'tertiary':
        rate_yield *= 1
    elif carbon_type != 'secondary':
        raise ValueError('Only primary, secondary and tertiary accepted for carbon_type')
    if other_functional_groups is not None:
        for fg in other_functional_groups:
            if (fg == 'hydroxy') or (fg == 'OH'):
                rate_yield *= 0.65 #for the equivalent group of atkinson 1997 for hydroxy groups formed from alkenes
    return rate_yield

In [ ]:
data_jenkins = np.ndarray((len(temperatures),len(pressures)))
data_alkoxy_jenkins = np.ndarray((len(temperatures),len(pressures)))
for i1, t in enumerate(temperatures):
    for i2, p in enumerate(pressures):
        nitrate_yield = get_nitrate_yield_jenkins(number_carbons,p,t,carbon_type,other_functional_groups=other_functional_groups)
        if use_total_rate:
            total_rate = total_kinetics.get_rate_coefficient(t)
            data_jenkins[i1,i2] = total_rate * nitrate_yield
            data_alkoxy_jenkins[i1,i2] = total_rate * (1-nitrate_yield)
        else:
            nitrate_to_NO2_ratio = 1 / (1 / nitrate_yield - 1)
            data_jenkins[i1,i2] = alkoxy_kinetics.get_rate_coefficient(t) * nitrate_to_NO2_ratio

# use RMG to get plog of data

In [ ]:
temperatures = np.linspace(250,600,20)
pressures = np.logspace(3,7,5)

In [ ]:
from rmgpy.kinetics.arrhenius import Arrhenius, PDepArrhenius
from rmgpy.chemkin import write_kinetics_entry
from rmgpy.reaction import Reaction
from rmgpy.species import Species

## n-propane peroxy radical

In [ ]:
number_carbons = 3
carbon_type = 'primary'
peroxy_name = 'npropyloo'
nitrate_name = 'npropylONO2'
alkoxy_name = 'npropyloxy'
peroxy_smiles = 'CCCO[O]'
nitrate_smiles = 'CCCO[N+](=O)[O-]'
alkoxy_smiles = 'CCC[O]'
use_total_rate = False
other_functional_groups = []

In [ ]:
# get the alkoxy rates
if use_total_rate:
    #atkinson 1997 for C2+
    total_kinetics = Arrhenius(A=(2.7e-12*6.022e23,'cm^3/(mol*s)'),n=0,Ea=(-360*8.314,('J/mol')))
else:
    #anderlohr 2009
    alkoxy_kinetics = Arrhenius(A=(4.7e12,'cm^3/(mol*s)'),n=0,Ea=(-358,('cal/mol')))

In [ ]:
data = np.ndarray((len(temperatures),len(pressures)))
data_alkoxy = np.ndarray((len(temperatures),len(pressures)))
for i1, t in enumerate(temperatures):
    for i2, p in enumerate(pressures):
        nitrate_yield = get_nitrate_yield_jenkins(number_carbons,p,t,carbon_type,other_functional_groups=other_functional_groups)
        if use_total_rate:
            total_rate = total_kinetics.get_rate_coefficient(t)
            data[i1,i2] = total_rate * nitrate_yield
            data_alkoxy[i1,i2] = total_rate * (1-nitrate_yield)
        else:
            nitrate_to_NO2_ratio = 1 / (1 / nitrate_yield - 1)
            data[i1,i2] = alkoxy_kinetics.get_rate_coefficient(t) * nitrate_to_NO2_ratio

In [ ]:
nitrate_rate = PDepArrhenius().fit_to_data(temperatures,pressures, data, 'm^3/(mol*s)')
if use_total_rate:
    alkoxy_rate = PDepArrhenius().fit_to_data(temperatures,pressures, data_alkoxy, 'm^3/(mol*s)')

In [129]:
no = Species().from_smiles('[N]=O')
peroxy = Species(label = peroxy_name).from_smiles(peroxy_smiles)
nitrate = Species(label = nitrate_name).from_smiles(nitrate_smiles)

rxn = Reaction(reactants = [no,peroxy], products = [nitrate], kinetics = nitrate_rate)
print(write_kinetics_entry(rxn, [no,peroxy,nitrate]))
# this is in units of kcal/mol and mol and may need to have the activation energy modified

NO+npropyloo=npropylONO2                            1.000e+00 0.000     0.000    
    PLOG/ 0.009869  4.502e+23 -5.034    2.038    /
    PLOG/ 0.098692  1.440e+34 -8.458    3.538    /
    PLOG/ 0.986923  3.522e+33 -8.262    2.367    /
    PLOG/ 9.869233  1.109e+44 -11.810   4.012    /
    PLOG/ 98.692327 1.066e+40 -10.589   2.248    /



In [130]:
if use_total_rate:
    alkoxy = Species(label = alkoxy_name).from_smiles(alkoxy_smiles)
    no2 = Species().from_smiles('N(=O)[O]')
    rxn2 = Reaction(reactants = [no,peroxy], products = [alkoxy, no2], kinetics = alkoxy_rate)
    print(write_kinetics_entry(rxn2, [no,peroxy,alkoxy,no2]))
    # this is in units of kcal/mol and mol and may need to have the activation energy modified

## i-propane peroxy radical

In [ ]:
number_carbons = 3
carbon_type = 'secondary'
peroxy_name = 'ipropyloo'
nitrate_name = 'ipropylONO2'
alkoxy_name = 'ipropyloxy'
peroxy_smiles = 'CC(C)O[O]'
nitrate_smiles = 'CC(C)O[N+](=O)[O-]'
alkoxy_smiles = 'CC(C)[O]'
use_total_rate = False
other_functional_groups = []

In [ ]:
# get the alkoxy rates
if use_total_rate:
    #atkinson 1997 for C2+
    total_kinetics = Arrhenius(A=(2.7e-12*6.022e23,'cm^3/(mol*s)'),n=0,Ea=(-360*8.314,('J/mol')))
else:
    #anderlohr 2009
    alkoxy_kinetics = Arrhenius(A=(4.7e12,'cm^3/(mol*s)'),n=0,Ea=(-358,('cal/mol')))

In [ ]:
data = np.ndarray((len(temperatures),len(pressures)))
data_alkoxy = np.ndarray((len(temperatures),len(pressures)))
for i1, t in enumerate(temperatures):
    for i2, p in enumerate(pressures):
        nitrate_yield = get_nitrate_yield_jenkins(number_carbons,p,t,carbon_type,other_functional_groups=other_functional_groups)
        if use_total_rate:
            total_rate = total_kinetics.get_rate_coefficient(t)
            data[i1,i2] = total_rate * nitrate_yield
            data_alkoxy[i1,i2] = total_rate * (1-nitrate_yield)
        else:
            nitrate_to_NO2_ratio = 1 / (1 / nitrate_yield - 1)
            data[i1,i2] = alkoxy_kinetics.get_rate_coefficient(t) * nitrate_to_NO2_ratio

In [ ]:
nitrate_rate = PDepArrhenius().fit_to_data(temperatures,pressures, data, 'm^3/(mol*s)')
if use_total_rate:
    alkoxy_rate = PDepArrhenius().fit_to_data(temperatures,pressures, data_alkoxy, 'm^3/(mol*s)')

In [135]:
no = Species().from_smiles('[N]=O')
peroxy = Species(label = peroxy_name).from_smiles(peroxy_smiles)
nitrate = Species(label = nitrate_name).from_smiles(nitrate_smiles)

rxn = Reaction(reactants = [no,peroxy], products = [nitrate], kinetics = nitrate_rate)
print(write_kinetics_entry(rxn, [no,peroxy,nitrate]))
# this is in units of kcal/mol and mol and may need to have the activation energy modified

NO+ipropyloo=ipropylONO2                            1.000e+00 0.000     0.000    
    PLOG/ 0.009869  6.938e+23 -5.034    2.038    /
    PLOG/ 0.098692  2.196e+34 -8.457    3.535    /
    PLOG/ 0.986923  3.395e+33 -8.198    2.297    /
    PLOG/ 9.869233  2.047e+43 -11.516   3.716    /
    PLOG/ 98.692327 1.049e+37 -9.563    1.281    /



In [ ]:
if use_total_rate:
    alkoxy = Species(label = alkoxy_name).from_smiles(alkoxy_smiles)
    no2 = Species().from_smiles('N(=O)[O]')
    rxn2 = Reaction(reactants = [no,peroxy], products = [alkoxy, no2], kinetics = alkoxy_rate)
    print(write_kinetics_entry(rxn2, [no,peroxy,alkoxy,no2]))
    # this is in units of kcal/mol and mol and may need to have the activation energy modified

## butane peroxy radical

In [ ]:
number_carbons = 4
carbon_type = 'primary'
peroxy_name = 'PC4H9O2'
nitrate_name = 'PC4H9ONO2'
alkoxy_name = 'PC4H9O'
peroxy_smiles = 'CCCCO[O]'
nitrate_smiles = 'CCCCO[N+](=O)[O-]'
alkoxy_smiles = 'CCCC[O]'
use_total_rate = True
other_functional_groups = []

In [ ]:
# get the alkoxy rates
if use_total_rate:
    #atkinson 1997 for C2+
    total_kinetics = Arrhenius(A=(2.7e-12*6.022e23,'cm^3/(mol*s)'),n=0,Ea=(-360*8.314,('J/mol')))
else:
    #anderlohr 2009
    alkoxy_kinetics = Arrhenius(A=(4.7e12,'cm^3/(mol*s)'),n=0,Ea=(-358,('cal/mol')))

In [ ]:
data = np.ndarray((len(temperatures),len(pressures)))
data_alkoxy = np.ndarray((len(temperatures),len(pressures)))
for i1, t in enumerate(temperatures):
    for i2, p in enumerate(pressures):
        nitrate_yield = get_nitrate_yield_jenkins(number_carbons,p,t,carbon_type,other_functional_groups=other_functional_groups)
        if use_total_rate:
            total_rate = total_kinetics.get_rate_coefficient(t)
            data[i1,i2] = total_rate * nitrate_yield
            data_alkoxy[i1,i2] = total_rate * (1-nitrate_yield)
        else:
            nitrate_to_NO2_ratio = 1 / (1 / nitrate_yield - 1)
            data[i1,i2] = alkoxy_kinetics.get_rate_coefficient(t) * nitrate_to_NO2_ratio

In [ ]:
nitrate_rate = PDepArrhenius().fit_to_data(temperatures,pressures, data, 'm^3/(mol*s)')
if use_total_rate:
    alkoxy_rate = PDepArrhenius().fit_to_data(temperatures,pressures, data_alkoxy, 'm^3/(mol*s)')

In [111]:
no = Species().from_smiles('[N]=O')
peroxy = Species(label = peroxy_name).from_smiles(peroxy_smiles)
nitrate = Species(label = nitrate_name).from_smiles(nitrate_smiles)

rxn = Reaction(reactants = [no,peroxy], products = [nitrate], kinetics = nitrate_rate)
print(write_kinetics_entry(rxn, [no,peroxy,nitrate]))
# this is in units of kcal/mol and mol and may need to have the activation energy modified

NO+PC4H9O2=PC4H9ONO2                                1.000e+00 0.000     0.000    
    PLOG/ 0.009869  2.369e+30 -7.373    3.009    /
    PLOG/ 0.098692  8.428e+31 -7.866    2.299    /
    PLOG/ 0.986923  5.591e+38 -10.148   3.057    /
    PLOG/ 9.869233  2.035e+45 -12.405   3.852    /
    PLOG/ 98.692327 1.110e+43 -11.709   2.905    /



In [112]:
if use_total_rate:
    alkoxy = Species(label = alkoxy_name).from_smiles(alkoxy_smiles)
    no2 = Species().from_smiles('N(=O)[O]')
    rxn2 = Reaction(reactants = [no,peroxy], products = [alkoxy, no2], kinetics = alkoxy_rate)
    print(write_kinetics_entry(rxn2, [no,peroxy,alkoxy,no2]))
    # this is in units of kcal/mol and mol and may need to have the activation energy modified

NO+PC4H9O2=PC4H9O+NO2                               1.000e+00 0.000     0.000    
    PLOG/ 0.009869  1.613e+12 0.001     -0.715   /
    PLOG/ 0.098692  1.912e+12 -0.021    -0.685   /
    PLOG/ 0.986923  9.348e+12 -0.241    -0.465   /
    PLOG/ 9.869233  3.899e+14 -0.758    0.047    /
    PLOG/ 98.692327 1.316e+17 -1.573    0.792    /



## gamma-isobutanol peroxy radical

In [ ]:
number_carbons = 5
carbon_type = 'primary'
peroxy_name = 'gRO2'
nitrate_name = 'NO2OCC(CO)C'
alkoxy_name = '[O]CC(CO)C'
peroxy_smiles = 'CC(CO)CO[O]'
nitrate_smiles = 'CC(CO)CO[N+](=O)[O-]'
alkoxy_smiles = 'CC(C)C[O]'
use_total_rate = True
other_functional_groups = ['OH']

In [ ]:
# get the alkoxy rates
if use_total_rate:
    #atkinson 1997 for C2+
    total_kinetics = Arrhenius(A=(2.7e-12*6.022e23,'cm^3/(mol*s)'),n=0,Ea=(-360*8.314,('J/mol')))
else:
    #anderlohr 2009
    alkoxy_kinetics = Arrhenius(A=(4.7e12,'cm^3/(mol*s)'),n=0,Ea=(-358,('cal/mol')))

In [ ]:
data = np.ndarray((len(temperatures),len(pressures)))
data_alkoxy = np.ndarray((len(temperatures),len(pressures)))
for i1, t in enumerate(temperatures):
    for i2, p in enumerate(pressures):
        nitrate_yield = get_nitrate_yield_jenkins(number_carbons,p,t,carbon_type,other_functional_groups=other_functional_groups)
        if use_total_rate:
            total_rate = total_kinetics.get_rate_coefficient(t)
            data[i1,i2] = total_rate * nitrate_yield
            data_alkoxy[i1,i2] = total_rate * (1-nitrate_yield)
        else:
            nitrate_to_NO2_ratio = 1 / (1 / nitrate_yield - 1)
            data[i1,i2] = alkoxy_kinetics.get_rate_coefficient(t) * nitrate_to_NO2_ratio

In [ ]:
nitrate_rate = PDepArrhenius().fit_to_data(temperatures,pressures, data, 'm^3/(mol*s)')
if use_total_rate:
    alkoxy_rate = PDepArrhenius().fit_to_data(temperatures,pressures, data_alkoxy, 'm^3/(mol*s)')

In [117]:
no = Species().from_smiles('[N]=O')
peroxy = Species(label = peroxy_name).from_smiles(peroxy_smiles)
nitrate = Species(label = nitrate_name).from_smiles(nitrate_smiles)

rxn = Reaction(reactants = [no,peroxy], products = [nitrate], kinetics = nitrate_rate)
print(write_kinetics_entry(rxn, [no,peroxy,nitrate]))
# this is in units of kcal/mol and mol and may need to have the activation energy modified

NO+gRO2=NO2OCC(CO)C                                 1.000e+00 0.000     0.000    
    PLOG/ 0.009869  4.428e+33 -8.505    3.364    /
    PLOG/ 0.098692  1.836e+32 -8.043    2.009    /
    PLOG/ 0.986923  7.693e+43 -11.940   4.002    /
    PLOG/ 9.869233  5.451e+43 -11.980   3.270    /
    PLOG/ 98.692327 5.494e+42 -11.675   2.827    /



In [118]:
if use_total_rate:
    alkoxy = Species(label = alkoxy_name).from_smiles(alkoxy_smiles)
    no2 = Species().from_smiles('N(=O)[O]')
    rxn2 = Reaction(reactants = [no,peroxy], products = [alkoxy, no2], kinetics = alkoxy_rate)
    print(write_kinetics_entry(rxn2, [no,peroxy,alkoxy,no2]))
    # this is in units of kcal/mol and mol and may need to have the activation energy modified

NO+gRO2=C4H9O+NO2                                   1.000e+00 0.000     0.000    
    PLOG/ 0.009869  1.628e+12 0.000     -0.713   /
    PLOG/ 0.098692  2.396e+12 -0.053    -0.656   /
    PLOG/ 0.986923  9.699e+12 -0.246    -0.460   /
    PLOG/ 9.869233  2.642e+14 -0.706    -0.022   /
    PLOG/ 98.692327 2.303e+15 -1.009    0.256    /

